# Background:
In this assignment, we will evaluate the performance of our 'Active Portfolio' (Portfolio constructed using the same Instruments, strategies and MVA weights as used in Part A as our 'Final Portfolio') on an 'Out of Sample' timeframe starting from 1/1/2019 and ending on 10/31/2022. Then, we will compare it with an 'Equal Weighted Portfolio' with no technical stratgeies (will be referred as 'Passive Portfolio'). The primary metric of evaluation of Portfolio will be 'Sharpe Ratio'.

In [1]:
# importing necessary libraries 
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.lines as mlines
import numpy as np
import pandas as pd
import scipy.stats as scs
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
from itertools import product
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

### Q1

### Importing the dataset
We'll import the new dataset, starting from 12/31/1999 (just like Part A) and ending on 10/31/2022. Then we'll create the 'out of sample' data by slicing the dataframe as per our required timeframe (that is, from Jan 1, 2019 to Oct 31, 2022). Then we'll look at the descriptive statistics of this dataframe.

In [2]:
#loading file into dataframe 
path = "PricesThru2022.csv"
prices=pd.read_csv(path,parse_dates=True)
prices.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
prices.set_index("Date",inplace=True)
prices.head()

,AAPL,AMZN,ATT,AUD,BOND,EUR,GE,GOLD,INTC,SILVER,SPY
Date,,,,,,,,,,,
1999-12-31,0.917969,3.806250,36.820244,0.6567,798.154631,1.0062,396.794861,288.00,41.15625,5.380,146.8750
2000-01-03,0.999442,4.468750,35.498489,0.6585,793.868670,1.0243,384.615387,289.00,43.50000,5.405,145.4375
2000-01-04,0.915179,4.096875,33.421452,0.6550,796.761677,1.0296,369.230774,282.25,41.46875,5.325,139.7500
2000-01-05,0.928571,3.487500,33.940708,0.6579,793.118215,1.0321,368.589752,280.00,41.81250,5.145,140.0000
2000-01-06,0.848214,3.278125,33.043808,0.6533,795.091984,1.0328,373.517639,281.10,39.37500,5.120,137.7500


In [3]:
prices = prices.loc['2019-01-01' : '2022-10-31']
prices

,AAPL,AMZN,ATT,AUD,BOND,EUR,GE,GOLD,INTC,SILVER,SPY
Date,,,,,,,,,,,
2019-01-02,39.480000,76.956497,22.311178,0.6985,1002.130122,1.1344,61.923077,1284.59,47.080002,15.5217,250.179993
2019-01-03,35.547501,75.014000,22.341391,0.7006,1007.528585,1.1394,62.000000,1294.28,44.490002,15.7415,244.210007
2019-01-04,37.064999,78.769501,22.915407,0.7113,1002.707612,1.1395,63.307693,1286.05,47.220001,15.6988,252.389999
2019-01-07,36.982498,81.475502,23.330816,0.7148,1002.282520,1.1474,67.230766,1289.21,47.439999,15.6535,254.380005
2019-01-08,37.687500,82.829002,23.625378,0.7140,1000.889790,1.1441,65.846153,1285.39,47.740002,15.6543,256.769989
...,...,...,...,...,...,...,...,...,...,...,...
2022-10-25,152.339996,120.599998,17.690001,0.6394,844.938544,0.9966,73.000000,1653.17,27.410000,19.3473,384.920013
2022-10-26,149.350006,115.660004,18.139999,0.6497,849.013953,1.0081,75.459999,1664.57,27.209999,19.5815,382.019989
2022-10-27,144.800003,110.959999,18.030001,0.6452,853.391436,0.9964,76.000000,1663.31,26.270000,19.6014,379.980011


In [4]:
#eyeballing prices dataframe
prices.describe()

,AAPL,AMZN,ATT,AUD,BOND,EUR,GE,GOLD,INTC,SILVER,SPY
count,959.000000,959.000000,959.000000,959.000000,959.000000,959.000000,959.000000,959.000000,959.000000,959.000000,959.000000
mean,109.362072,130.830561,22.371139,0.709737,1030.741791,1.128080,81.641216,1688.550104,51.164734,20.858597,360.458989
std,43.686852,33.087503,3.338179,0.038199,60.889745,0.057968,17.830810,202.453574,8.324322,4.249190,64.241785
min,35.547501,75.014000,14.630000,0.574300,837.016444,0.959400,43.919998,1270.690000,25.040001,11.981000,222.949997
25%,65.728752,94.959999,20.407854,0.686800,1010.626113,1.101500,68.159999,1516.630000,47.424999,17.142950,298.945007
50%,121.099998,134.643494,22.205439,0.710500,1049.520140,1.127700,81.599998,1755.840000,51.660000,21.114400,362.570007
75%,146.875000,161.498497,23.768883,0.733650,1068.515239,1.177100,96.825001,1837.595000,56.955000,24.624250,418.919998
max,182.009995,186.570496,29.932024,0.796800,1109.469563,1.232700,114.800003,2063.540000,68.470001,29.129500,477.709991


### Q2

#### Creating our 'Active Portfolio' (by using the same MVO weights distribution and selecting the same instruments as Part A). Then, evaluating its performance on the 'Out of Sample' dataframe with 'Sharpe Ratio' as metric.
<ol>
    <li> Create useful functions for implementing repititive processes efficiently. </li>
    <li> Slice the dataframe as we need only those instrument which we used in Part A and evaluate logReturns. </li>
     <li> Implement the same strategies on instruments as per Part A. </li>
    <li> Select the MVO Weights from Part A and use it to make the 'Active Portfolio' using weighted cummulative Returns. </li>
    <li> Evaluate the performance of our 'Active Portfolio' with Sharpe Ratio as our metric.   </li>
</ol>

In [5]:
# universal plot dimensions we will use as the figsize argument for all plots
aspect_ratio=(8,5)
pct_format_4d = '{:.4%}'
pct_format_2d = '{:.2%}'

In [6]:
# Creating a function to take in a dataframe and return it out in a percentage format rounded to 2 decimal placed
def print_data_frame_pct (dF, pct_format=pct_format_4d, nan='NaN', indexSlice=pd.IndexSlice[:,:]): # definging function name along with its arguments
    return (dF.style.format(pct_format, na_rep=nan, subset=indexSlice)) # displaying the dataframe in the required format

In [7]:
#function to calculate summary stats
annualization_factor=260
pct_format_2d = '{:.2%}'
def calc_summary_stats(dF, af=annualization_factor, pct_format=pct_format_2d):
    cumRtns = dF.cumsum()
    summary_stats = pd.DataFrame([af*dF.mean(),
                                  np.sqrt(af)*dF.std(),
                                  np.sqrt(af) * dF.mean() / dF.std(),
                                  (cumRtns - np.maximum.accumulate(cumRtns)).min(axis=0),
                                  (dF >= 0).sum() / dF.shape[0]
                                 ],
                                 index=['Average Annual Return', 'Annual Risk', 'Sharpe Ratio', 'maxDD', 'Success Ratio']
                                )
    print_data_frame_pct(summary_stats, pct_format=pct_format, indexSlice=pd.IndexSlice[['Average Annual Return', 'Annual Risk', 'maxDD', 'Success Ratio'], :])
    #percent_df_display(summary_stats, pct_format)
    return summary_stats

In [8]:
def pretty_print (dF, pct_format=pct_format_4d, nan='NaN', indexSlice=pd.IndexSlice[:,:]):
    display(dF.style.format(pct_format, na_rep=nan, subset=indexSlice))

In [9]:
#Function to pretty print the dataframe 
# data frame "pretty print" function to print percentages. There are some optional parameters
annualization_factor=260
pct_format_2d = '{:.2%}'
def calculate_summary_final(dF, af=annualization_factor, pct_format=pct_format_2d):
    cumRtns = dF.cumsum()
    summary_stats = pd.DataFrame([af*dF.mean(),
                                  np.sqrt(af)*dF.std(),
                                  np.sqrt(af) * dF.mean() / dF.std(),
                                  (cumRtns - np.maximum.accumulate(cumRtns)).min(axis=0),
                                  (dF >= 0).sum() / dF.shape[0]
                                 ],
                                 index=['Average Annual Return', 'Annual Risk', 'Sharpe Ratio', 'maxDD', 'Success Ratio']
                                )
    pretty_print(summary_stats, pct_format=pct_format, indexSlice=pd.IndexSlice[['Average Annual Return', 'Annual Risk', 'maxDD', 'Success Ratio'], :])
    #percent_df_display(summary_stats, pct_format)

In [10]:
#Portfolio with 8 instruments
df_prices = prices[['AAPL','AMZN','ATT','GE','INTC','GOLD','EUR','BOND']]
df_prices

,AAPL,AMZN,ATT,GE,INTC,GOLD,EUR,BOND
Date,,,,,,,,
2019-01-02,39.480000,76.956497,22.311178,61.923077,47.080002,1284.59,1.1344,1002.130122
2019-01-03,35.547501,75.014000,22.341391,62.000000,44.490002,1294.28,1.1394,1007.528585
2019-01-04,37.064999,78.769501,22.915407,63.307693,47.220001,1286.05,1.1395,1002.707612
2019-01-07,36.982498,81.475502,23.330816,67.230766,47.439999,1289.21,1.1474,1002.282520
2019-01-08,37.687500,82.829002,23.625378,65.846153,47.740002,1285.39,1.1441,1000.889790
...,...,...,...,...,...,...,...,...
2022-10-25,152.339996,120.599998,17.690001,73.000000,27.410000,1653.17,0.9966,844.938544
2022-10-26,149.350006,115.660004,18.139999,75.459999,27.209999,1664.57,1.0081,849.013953
2022-10-27,144.800003,110.959999,18.030001,76.000000,26.270000,1663.31,0.9964,853.391436


In [11]:
# defining instruments for short and flat statergy
prices_cols_short = ['ATT','GE']
prices_cols_flat = ['AAPL', 'AMZN', 'EUR', 'GOLD']

In [12]:
#Creating a function to get SMA Returns for the portfolio 
def getSMA(df, col ,fastWindow, slowWindow, flatOrShort=0):
    df.loc[:,'FastSMA'] = df[col].rolling(fastWindow).mean()
    df.loc[:,'SlowSMA'] = df[col].rolling(slowWindow).mean()
    df.dropna(inplace=True) # drop off all the Nulls created by taking SMAs
    df.loc[:,'Position'] = np.where(df['FastSMA'] > df['SlowSMA'], 1, flatOrShort) # Stay long till Fast SMA is > Slow SMA, else go flat
    df.loc[:,'OriginalReturns'] = np.log(df[col] / df[col].shift(1))  # calculate SPY stock price log returns
    df.loc[:,'SMAReturns'] = df['Position'].shift(1) * df['OriginalReturns']  # calculte the log SMA returns, by pushing down the Position column
    df.dropna(inplace=True) # drop off all the remaining Nulls
    #return calc_summary_stats(df[['OriginalReturns', 'SMAReturns']]) # return only the required 2 columns
    return df['SMAReturns']



In [13]:
#creating bollinger band function by accepting dataframe, lookback window and std band as inputs and returns orginal and BB returns
def Bollinger_Bands(df,col,window,stDevBand):
    dfPrices = df.copy()
    dfPrices = dfPrices[[col]]
    dfPrices.loc[:,'Mean'] = dfPrices[col].rolling(window).mean()
    dfPrices.loc[:,'Stdev'] = dfPrices[col].rolling(window).std()
    dfPrices.loc[:,'Upper'] = dfPrices['Mean'] + stDevBand * dfPrices['Stdev']
    dfPrices.loc[:,'Lower'] = dfPrices['Mean'] - stDevBand * dfPrices['Stdev']
    dfPrices.dropna(inplace=True)
    date0 = dfPrices.index[0]
    if dfPrices.loc[date0,col] >= dfPrices.loc[date0, 'Upper']:
        dfPrices.loc[date0, 'Position'] = -1
    elif dfPrices.loc[date0,col] <= dfPrices.loc[date0,'Lower']:
        dfPrices.loc[date0, 'Position'] = 1
    else:
        dfPrices.loc[date0, 'Position'] = 0

# then loop over the entire data set
    for i in range(1, dfPrices.shape[0]):
        today=dfPrices.index[i]
        yesterday=dfPrices.index[i-1]
        if dfPrices.loc[today,col] >= dfPrices.loc[today,'Upper']: # if close is above upper
            dfPrices.loc[today,'Position'] = -1 # then go short
        elif dfPrices.loc[today,col] <= dfPrices.loc[today,'Lower']: # if close is below lower
            dfPrices.loc[today,'Position'] = 1 # then go long
        elif dfPrices.loc[yesterday,'Position'] == -1 and dfPrices.loc[today,col] <= dfPrices.loc[today,'Mean']: # if prev day is short and we're now below the mean
            dfPrices.loc[today,'Position']=0 # then flatten
        elif dfPrices.loc[yesterday,'Position'] == 1 and dfPrices.loc[today, col] >= dfPrices.loc[today, 'Mean']: # conversely...
            dfPrices.loc[today,'Position']=0 # then also flatten
        else: # otherwise just hold yesterday's position
            dfPrices.loc[today,'Position']=dfPrices.loc[yesterday,'Position']
    dfPrices.loc[:,'OriginalReturns'] = np.log(dfPrices[col] / dfPrices[col].shift(1))
    dfPrices.loc[:,'BBReturns'] = dfPrices['Position'].shift(1) * dfPrices['OriginalReturns']
    dfPrices.dropna(inplace=True)
    dfPrices[['OriginalReturns', 'BBReturns']].sum()
    np.exp(dfPrices[['OriginalReturns', 'BBReturns']].sum())
    dfPrices[['OriginalReturns', 'BBReturns']].mean() * 260
    dfPrices[['OriginalReturns', 'BBReturns']].std() * np.sqrt(260)
    return dfPrices['BBReturns']

In [14]:
#Running SMA flat for  'AAPL' , 'AMZN', 'EUR', 'GOLD' and and short statergies for ATT and GE
# For Bond we take the instrument itself
df_sma = pd.DataFrame()
for column in prices_cols_short:
    if column not in ['BOND','INTC']:
        flatOrShort = -1
        dfPrice2 = prices.copy()
        df_sma[column] = getSMA(dfPrice2, column, 42, 260, flatOrShort) # creating the dataframe using the mving average crossover function created above 
for column in prices_cols_flat:
    if column not in ['BOND','INTC']:
        dfPrice2 = prices.copy()
        df_sma[column] = getSMA(dfPrice2, column, 42,260, 0)

In [15]:
#Calculating log returns
logReturns = np.log(prices / prices.shift(1))
logReturns

,AAPL,AMZN,ATT,AUD,BOND,EUR,GE,GOLD,INTC,SILVER,SPY
Date,,,,,,,,,,,
2019-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-03,-0.104924,-0.025566,0.001353,0.003002,0.005373,0.004398,0.001241,0.007515,-0.056584,0.014061,-0.024152
2019-01-04,0.041803,0.048851,0.025368,0.015157,-0.004796,0.000088,0.020872,-0.006379,0.059553,-0.002716,0.032947
2019-01-07,-0.002228,0.033777,0.017966,0.004909,-0.000424,0.006909,0.060124,0.002454,0.004648,-0.002890,0.007854
2019-01-08,0.018884,0.016476,0.012546,-0.001120,-0.001391,-0.002880,-0.020810,-0.002967,0.006304,0.000051,0.009351
...,...,...,...,...,...,...,...,...,...,...,...
2022-10-25,0.019153,0.006489,0.010227,0.012907,0.009420,0.009274,-0.004919,0.002053,0.008427,0.005983,0.015842
2022-10-26,-0.019822,-0.041824,0.025120,0.015980,0.004812,0.011473,0.033143,0.006872,-0.007323,0.012032,-0.007563
2022-10-27,-0.030939,-0.041485,-0.006082,-0.006950,0.005143,-0.011674,0.007131,-0.000757,-0.035157,0.001016,-0.005354


In [16]:
#dropping na from log returns
logReturns.dropna(inplace=True)

In [17]:
#Running Bollinger Band strategy for Intel 
df_intc=pd.DataFrame()
df_intc['INTC']=Bollinger_Bands(df_prices,'INTC',20,2)

In [18]:
#inner join Intel returns with ma returns to match dates 
df_sma = df_sma.join(df_intc,how='inner')

In [19]:
#Joinning log returns for BOND
df_sma = df_sma.join(logReturns['BOND'], how='inner')

In [20]:
#Optimum weights for max sharp portfolio 
init_weights=np.array([0.0065161 , 0.09944205, 0.20441047, 0.05964576, 0.20504819,
        0.18976544, 0.06052765, 0.17464435])

In [21]:
#multiplying the max weights with daily rebalancing to create max sharpe portfolio
weightedCumReturns = init_weights* df_sma
weightedCumReturns

,ATT,GE,AAPL,AMZN,EUR,GOLD,INTC,BOND
Date,,,,,,,,
2020-01-16,0.000027,-0.000252,0.002545,0.000508,-0.0,-0.000457,0.000000,-0.000122
2020-01-17,0.000060,-0.000252,0.002251,-0.000421,-0.0,0.000577,-0.000000,-0.000188
2020-01-21,0.000024,-0.001271,-0.001390,0.000866,-0.0,0.000113,0.000000,0.000556
2020-01-22,0.000087,-0.002505,0.000728,-0.000143,0.0,0.000074,0.000000,0.000077
2020-01-23,-0.000069,0.003438,0.000982,-0.000091,-0.0,0.000506,-0.000567,0.000220
...,...,...,...,...,...,...,...,...
2022-10-25,-0.000067,0.000489,0.000000,0.000000,0.0,0.000000,0.000000,0.001645
2022-10-26,-0.000164,-0.003296,-0.000000,-0.000000,0.0,0.000000,-0.000000,0.000840
2022-10-27,0.000040,-0.000709,-0.000000,-0.000000,-0.0,-0.000000,-0.000000,0.000898


In [22]:
weightedCumReturns['Portfolio']=weightedCumReturns.sum(axis=1)

In [23]:
#checking if the portfolio is returning same sharp as before 
calculate_summary_final(weightedCumReturns)

,ATT,GE,AAPL,AMZN,EUR,GOLD,INTC,BOND,Portfolio
Average Annual Return,-0.13%,-2.76%,3.85%,0.90%,0.41%,0.47%,1.52%,-1.43%,2.84%
Annual Risk,0.19%,4.74%,7.33%,1.68%,0.78%,2.71%,2.19%,0.96%,12.27%
Sharpe Ratio,-0.675156,-0.583265,0.525067,0.537793,0.531225,0.173425,0.694965,-1.478943,0.231161
maxDD,-0.38%,-11.26%,-7.71%,-1.71%,-1.04%,-4.13%,-2.23%,-4.92%,-18.20%
Success Ratio,50.93%,48.07%,57.37%,65.67%,81.69%,73.68%,74.39%,49.50%,51.93%


### Q3

#### Now, let's compare the performance of our 'Active Portfolio' with that of 'Equal Weighted Portfolio' on our 'out of sample dataset' with Sharpe Ratio as our main evalution metric.
<ul>
    <li> The Equal weighted Portfolio will not be containing any technical strategies and will be referred as 'Passive Portfolio'</li>
</ul>

In [24]:
#defining equal weights for the equal weights portfolio
equal_wts = [0.125]*8

In [25]:
equal_wts=np.array(equal_wts)

In [26]:
#creating a deep copy of the portfolio dataframe 
logReturns_copy = logReturns.copy()

In [27]:
#logreturns 8 instruments for portfolio  
logReturns_copy = logReturns_copy[['AAPL','AMZN','ATT','GE','INTC','GOLD','EUR','BOND']]

In [28]:
#creating weights column to multiply with portfolio returns
logReturns_copy['Weights'] = [equal_wts] * len(logReturns_copy)

In [29]:
#multiuplying weights with daily rebalancing 
portfolio_2 = []
for i in range(len(logReturns_copy)):
    portfolio_2.append(np.dot(logReturns_copy['Weights'][i], logReturns_copy.iloc[i,:-1]))

In [30]:
#Adding portfolio returns column
logReturns_copy['Portfolio'] = portfolio_2

In [31]:
calculate_summary_final(logReturns_copy.drop('Weights',axis=1))

,AAPL,AMZN,ATT,GE,INTC,GOLD,EUR,BOND,Portfolio
Average Annual Return,36.83%,7.76%,-5.48%,6.20%,-13.69%,6.52%,-3.74%,-4.55%,3.73%
Annual Risk,34.93%,35.79%,27.21%,45.91%,40.03%,15.40%,7.29%,5.05%,17.29%
Sharpe Ratio,1.054378,0.216918,-0.201520,0.135016,-0.341972,0.423441,-0.513919,-0.899719,0.215783
maxDD,-37.73%,-60.08%,-71.59%,-87.43%,-100.59%,-24.05%,-25.07%,-28.18%,-34.70%
Success Ratio,54.07%,52.30%,52.61%,49.37%,50.10%,54.07%,49.90%,51.15%,54.59%


### Q4

#### Now, we would consider 'Equal Weighted Portfolio' as our 'Passive Portfolio' and compare it with the performance of our 'Active Portfolio' for the 'Out of Sample' test period. Sharpe Ratio would be our primary metric to compare the performance.

In [32]:
#final returns of the portfolio 
calculate_summary_final(logReturns_copy.drop('Weights',axis=1), pct_format=pct_format_2d)

,AAPL,AMZN,ATT,GE,INTC,GOLD,EUR,BOND,Portfolio
Average Annual Return,36.83%,7.76%,-5.48%,6.20%,-13.69%,6.52%,-3.74%,-4.55%,3.73%
Annual Risk,34.93%,35.79%,27.21%,45.91%,40.03%,15.40%,7.29%,5.05%,17.29%
Sharpe Ratio,1.054378,0.216918,-0.201520,0.135016,-0.341972,0.423441,-0.513919,-0.899719,0.215783
maxDD,-37.73%,-60.08%,-71.59%,-87.43%,-100.59%,-24.05%,-25.07%,-28.18%,-34.70%
Success Ratio,54.07%,52.30%,52.61%,49.37%,50.10%,54.07%,49.90%,51.15%,54.59%


In [33]:
#final returns of the active portfolio 
calculate_summary_final((weightedCumReturns), pct_format=pct_format_2d)

,ATT,GE,AAPL,AMZN,EUR,GOLD,INTC,BOND,Portfolio
Average Annual Return,-0.13%,-2.76%,3.85%,0.90%,0.41%,0.47%,1.52%,-1.43%,2.84%
Annual Risk,0.19%,4.74%,7.33%,1.68%,0.78%,2.71%,2.19%,0.96%,12.27%
Sharpe Ratio,-0.675156,-0.583265,0.525067,0.537793,0.531225,0.173425,0.694965,-1.478943,0.231161
maxDD,-0.38%,-11.26%,-7.71%,-1.71%,-1.04%,-4.13%,-2.23%,-4.92%,-18.20%
Success Ratio,50.93%,48.07%,57.37%,65.67%,81.69%,73.68%,74.39%,49.50%,51.93%


#### Inferences:
<ul>
    <li> From the above two summary stats tables we can see that sharpe ratio of our constructed (active) portfolio for the out of sample period: 1st Jan 2019- 31st Oct 2022 is 0.231 and the equal weighted (passive) portfolio's sharpe is 0.215. This shows that our constructed portfolio delivers 1.069 times better sharpe than the passive portfolio</li>
    <li> In comparision, for the previous time period 31st Dec 1999- 31st Dec 2018 the constructed portfolio (active)(sharpe: 1.14) delivered 2 times better sharpe than the passive weighted portfolio's sharpe of 0.576 which is better than the current
returns for the out of sample period. This could be because of the sharp fall and recovery during 2020 because of which the short statergy for ATT and GE in the active portfolio delivered lesser sharp than the benchmarks.
    </li>
  
</ul>
    